In [1]:
import pandas as pd
import openpyxl

In [131]:
# pip install openpyxl pandas

In [499]:

d = pd.read_excel('2022_raw.xlsx', engine='openpyxl')

#selec  related colums
# df = d.iloc[:, [8,17,20,24,25] ] #econ
df = d.iloc[:, [8,28,30,51,52,55,57,58,63,65] ] #discrim
# df= d.iloc[:, [8] + list(range(32, 36))+list(range(37, 42))] #threat

#coun frequencies
counts = {}
first_col = df.columns[0] ##Федеральный округ
for col in df.columns[1:]:
    counts[col] = df.groupby(first_col)[col].value_counts()#.fillna(0)

# # transform to data frame
counts=pd.DataFrame(counts)
counts['All'] = counts.iloc[:, 1:].sum(axis=1)


districts = df[first_col].unique()
district_percentages = {}
district_df_percentages = {}
district_df_count = {}
district_count = {}
distrcict_all={}
all_districts_yes = [0] * len(counts.columns)
all_districts_no = [0] * len(counts.columns)

# calculate percentages
for district in districts:
    total_yes = counts.loc[(district, 'Да'), 'All'] if (district, 'Да') in counts.index else 0
    total_no = counts.loc[(district, 'Нет'), 'All'] if (district, 'Нет') in counts.index else 0

#     print(total_yes)
#     print(total_no)
    if total_yes + total_no != 0: 
        district_percentages[district] =  (total_yes / (total_yes + total_no)) * 100
    else:
        district_percentages[district] = 0

    for col in counts.columns:
        da_count = counts.loc[(district, 'Да'), col] if (district, 'Да') in counts.index else 0
        net_count = counts.loc[(district, 'Нет'), col] if (district, 'Нет') in counts.index else 0
        
        if da_count + net_count != 0: 
            district_df_percentages[(district, col)] = (da_count / (da_count + net_count)) * 100
        elif (district, 'Да') not in counts.index:
            district_df_percentages[(district, col)] = 0
           
        else:
            district_df_percentages[(district, col)] = 100

#     print(data.loc[(district, 'Да')])
#     all_districts_yes += counts.loc[(district, 'Да')].values #if (district, 'Да') in data.index else [0] * len(data.columns)
#     all_districts_no += counts.loc[(district, 'Нет')].values #if (district, 'Нет') in data.index else [0] * len(data.columns)

    
# transform to data frame
# df_district_percentages = pd.DataFrame(list(district_percentages.items()), columns=['District', 'Percentage'])
df_district_df_percentages = pd.DataFrame(list(district_df_percentages.items()), columns=['District_Column', 'Percentage'])
df_district_df_percentages[['District', 'Column']] = pd.DataFrame(df_district_df_percentages['District_Column'].tolist(), 
                                                                  index=df_district_df_percentages.index)
df_district_df_percentages = df_district_df_percentages.pivot(index='District', columns='Column', values='Percentage')

# df_percentages = all_districts_yes/(all_districts_yes+all_districts_no)*100
# new_row = pd.Series(df_percentages, index=df_district_df_percentages.columns, name="Все")
# df_district_df_percentages = df_district_df_percentages.append(new_row)
averages = df_district_df_percentages.mean()
df_district_df_percentages.loc['Все'] = averages


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [500]:
df_district_df_percentages

Column,All,Были ли у вас в 2022 году какие-либо проблемы при поиске места работы/учебы из-за сексуальной ориентации и/или трансгендерности?,Приходилось ли вам в 2022 году сталкиваться с насильственными/ дискриминирующими действиями врачей на почве гомофобии/трансфобии?,Приходилось ли вам в 2022 году сталкиваться с отказом в предоставлении медицинских услуг из-за вашей сексуальной ориентации и/или трансгендерностии?,Приходилось ли вам в 2022 году сталкиваться со случаями отказа в доступе к государственным услугам из-за сексуальной ориентации и/или трансгендерности?,Приходилось ли вам в 2022 году терять работу/место учёбы из-за сексуальной ориентации и/или трансгендерности?,"Сталкивались ли вы в 2022 году с ситуацией, когда вас лишили жилья из-за вашей сексуальной ориентации и/или трансгендерности?","Сталкивались ли вы в 2022 году с трудностями в поиске жилья в связи с вашей сексуальной ориентацией, трансгендерностью или гендерной неконформностью?",Сталкивались ли вы в 2022 году с трудностями при взаимодействии с детскими образовательными или медицинскими учреждениями из-за ваших сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы в 2022 году с трудностями при взаимодействии с органами опеки из-за ваших сексуальной ориентации и/или трансгендерности?
District,,,,,,,,,,
Дальневосточный федеральный округ,6.727480,32.500000,34.375000,9.375000,2.970297,7.567568,5.445545,5.940594,18.181818,NaN
Приволжский федеральный округ,4.303227,18.446602,15.789474,4.678363,0.778643,4.566745,3.448276,6.562848,NaN,1.923077
Северо-Западный федеральный округ,4.674412,19.607843,17.330211,4.683841,1.754386,3.764861,2.819549,6.892231,6.504065,3.252033
Северо-Кавказский федеральный округ,8.163265,32.258065,50.000000,25.000000,NaN,14.285714,3.389831,11.864407,NaN,NaN
Сибирский федеральный округ,3.824092,23.507463,10.891089,5.940594,1.335559,3.846154,4.006678,4.841402,NaN,NaN
Уральский федеральный округ,4.896339,21.266968,19.767442,6.976744,1.353965,4.048583,4.061896,7.350097,8.000000,NaN
Центральный федеральный округ,5.255053,19.520852,19.379845,6.976744,2.185315,4.524469,3.321678,7.167832,10.365854,3.048780
Южный федеральный округ,5.469388,20.000000,18.367347,16.326531,1.433692,8.270677,3.225806,5.017921,NaN,8.333333
Все,5.414157,23.388474,23.237551,9.994727,1.687408,6.359346,3.714907,6.954666,10.762934,4.139306


In [501]:
df_district_df_percentages.to_csv('summary_discrim_percent.csv')

In [494]:
df_district_df_percentages.to_csv('summary_econ_percent.csv')

In [497]:
df_district_df_percentages.to_csv('summary_threat_percent.csv')

In [424]:
unique_values = {col: df_discrim[col].unique() for col in df_discrim.columns}
unique_values

{'Федеральный округ': array(['Сибирский федеральный округ', 'Центральный федеральный округ',
        'Уральский федеральный округ', 'Дальневосточный федеральный округ',
        'Приволжский федеральный округ',
        'Северо-Западный федеральный округ', 'Южный федеральный округ',
        'Северо-Кавказский федеральный округ'], dtype=object),
 'Были ли у вас в 2022 году какие-либо проблемы при поиске места работы/учебы из-за сексуальной ориентации и/или трансгендерности?': array(['Да', nan, 'Нет'], dtype=object),
 'Приходилось ли вам в 2022 году терять работу/место учёбы из-за сексуальной ориентации и/или трансгендерности?': array(['Нет', 'Затрудняюсь ответить', 'Да'], dtype=object),
 'Приходилось ли вам в 2022 году сталкиваться с отказом в предоставлении медицинских услуг из-за вашей сексуальной ориентации и/или трансгендерностии?': array(['Да', nan, 'Нет'], dtype=object),
 'Приходилось ли вам в 2022 году сталкиваться с насильственными/ дискриминирующими действиями врачей на почве гом

# Organs

In [441]:
df.columns[47]

'Расскажите нам о вашем опыте взаимодействия с правоохранительными органами подробнее. Как происходило общение? Было ли это результативно?'

In [447]:
d=df.iloc[:, [45,46,47, 48] ]
unique_values = {col: d[col].unique() for col in d.columns}
unique_values

{'Полиция приняла ваше заявление?': array([nan, 'Да', 'Нет', 'Заявление подавалось онлайн',
        'Юрист посоветовал не писать заявления, а оставить разъяснение или как-то так назвал',
        'Приняли, сказав что угрозы расправы и  побои без видеодоказательств не являются основанием для уголовного дела. ',
        'Передумала подавать заявление',
        'Ребят есть люди где обращения в полицию без смысла, вы на теретории России были, чтобы задовать такие глупые вопросы ',
        'Заявления пытаются отфутболить ',
        'Да, но на протяжении 7 лет, ни кто не найдет , якобы. Хотя все известны .',
        'Вроде бы да',
        'Приняла, но особых действий не предприняла. Они просто поговорили с человеком. Человек оказался трусом и перестал открыто меня преследовать. Но за спиной всем рассказывает про меня.',
        'Сотрудники которые должны быть за полицию, закрывали на это глаза и бездействовали',
        'Заявление не писал. Я вызвал полицию, когда меня окружали толпой'],
    

In [445]:
 # Organs
    df.iloc[:, [45,46,47, 48] ]

,Полиция приняла ваше заявление?,Каковы были последствия вашего обращения в полицию?,Расскажите нам о вашем опыте взаимодействия с правоохранительными органами подробнее. Как происходило общение? Было ли это результативно?,"Почему вы не стали обращаться в правоохранительные органы, после того как пострадали от преступления на почве гомофобии/трансфобии?"
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
6434,NaN,NaN,NaN,NaN
6435,NaN,NaN,NaN,NaN
6436,NaN,NaN,NaN,Вы не верите в эффективность обращения в полицию
6437,NaN,NaN,NaN,Вы не верите в эффективность обращения в полиц...


# Eco

In [438]:
df.columns

Index(['Отметка времени',
       'Как вы себя определяете (ваша гендерная идентичность)?',
       'Идентифицируете ли вы себя как трансгендерного человека? ',
       'Ваша сексуальная ориентация', 'Являетесь ли вы интерсекс-человеком?',
       'Ваш возраст', 'Где вы проживаете? ',
       'Выберите, пожалуйста, регион, в котором вы проживаете',
       'Федеральный округ',
       'Кто из ваших друзей знает о том, что вы ЛГБТК+ персона?',
       'Кто из ваших родных знает о том, что вы ЛГБТК+ персона?',
       'Кто из ваших одноклассников/однокурсников/коллег (в нынешнем учебном/рабочем коллективе) знает о том, что вы ЛГБТК+ персона?',
       'Ваш круг общения, в основном, состоит из...',
       'Ваш круг общения – это, в основном, (по вашим ощущениям)…',
       'Ваше образование',
       'Охарактеризуйте, пожалуйста, ваше общее материальное положение.',
       'Где вы преимущественно жили в 2022 году?',
       'Есть ли у вас иждивенцы - те, кто не имеет собственного источника дохода и ко

In [485]:
d.iloc[:, [17,20,24,25] ]

,"Есть ли у вас иждивенцы - те, кто не имеет собственного источника дохода и кого вы финансово поддерживаете на постоянной основе, например, дети, родственники, партнёр(ка)?",Есть ли у вас постоянный источник дохода?,"Есть ли у вас вспомогательные источники какой-либо финансовой поддержки (например, родные, партнер/ка, друзья регулярно или хотя бы иногда помогают вам материально)?","Приходилось ли вам в 2022 году получать оплату за секс, эскорт, работу моделью в порнографии или вебкам-моделью?"
0,Нет,Да,Нет,Нет
1,Нет,Нет,Да,Нет
2,Нет,Да,Да,Нет
3,Нет,Нет,Да,Нет
4,Нет,Нет,Да,Нет
...,...,...,...,...
6434,Нет,Да,Нет,Нет
6435,Нет,Нет,Да,Нет
6436,Да,Да,Нет,Да
6437,Нет,Нет,Да,Нет
